In [40]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas

In [2]:
nationalRailURL = 'https://ojp.nationalrail.co.uk/service/timesandfares'
#Order baseURL,departure station,arrival station,today(word),time,'/dep'
disleyCode = 'DSL'
middlewoodCode = 'MDL'
hazelGroveCode = 'HAZ'

Route:

   X'ing
     |
     v
HAZ -> MDL -> DSL

In [3]:
def urlRequestBulk():
    time = datetime.now().strftime("%H%M")
    urlConstOne = nationalRailURL + '/' + disleyCode + '/' + hazelGroveCode + '/' + 'today/' + time + '/dep#outwardJump'
    urlConstTwo = nationalRailURL + '/' + hazelGroveCode + '/' + disleyCode + '/' + 'today/' + time + '/dep#outwardJump'
    print(urlConstOne)
    print(urlConstTwo)
    DSLtoHAZ_html = requests.get(urlConstOne).text
    HAZtoDSL_html = requests.get(urlConstTwo).text
    return DSLtoHAZ_html, HAZtoDSL_html   

In [7]:
def tableDataExtraction(htmlData):
    extracted_tables = []
    for i in range(len(htmlData)):
        data = []
        html_Soup = BeautifulSoup(htmlData[i], 'html.parser')
        valueTable = html_Soup.find('table', attrs={'id' : 'oft'})
        table_body = valueTable.find('tbody')

        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])
        extracted_tables.append(data)

    return extracted_tables

In [47]:
def main():
    DSLtoHAZ, HAZtoDSL = urlRequestBulk()
    htmlData = [DSLtoHAZ, HAZtoDSL]
    tableData = tableDataExtraction(htmlData)
    for i in range (len(tableData)): #Clean-Up of HTML tables
        for e in range (len(tableData[i])):
            for a in range(len(tableData[i][e])):
                if 'Departs' or 'Arrives' in tableData[i][e][a]:
                    tableData[i][e][a] = tableData[i][e][a].replace('\n','').replace('\t','')
                else:
                    tableData[i][e].pop([a])
        print(tableData[i])

In [48]:
main()

https://ojp.nationalrail.co.uk/service/timesandfares/DSL/HAZ/today/1502/dep#outwardJump
https://ojp.nationalrail.co.uk/service/timesandfares/HAZ/DSL/today/1502/dep#outwardJump
[['Departs\xa0at15:27Disley [DSL]Platform2', '07m0 change(s)More view details  on the 15:27service from Disley toHazel Grove', 'Arrives\xa015:34Hazel Grove [HAZ]Platform2Buy from Select train operatorSoutheasternGreat Western RailwayGreater AngliaThameslinkGrand CentralRailEasyLNERHull Trainsthetrainline.comC2CGreat NorthernRedspottedhankyNorthernSouthernEast Midlands RailwayAvanti West CoastChiltern RailwaysCaledonian SleeperLondon Northwestern RailwayWest Midlands RailwayLumoCrossCountryScotRailSouth Western RailwayTransport for WalesTransPennine ExpressRailEasyRedspottedhankythetrainline.com', 'on time', 'cheapest fareSingle Fare£3.70Buy Now Anytime Othertickets'], ['Service UpdateStrike action by RMT union will significantly impact Northern services between 18 and 21 August.More information can be found here: